In [19]:
from datetime import datetime
import json
import os
from os.path import join as pjoin
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go


DATA_DIR = './data'


probes_names = []
probes_mean_times = []
config_dates = []
itk_versions = []

# TODO
# Look for a way to get data belonging to a [probes_names, itk_version] pair
# in a separate structure; may be enough using probes_names as a dictionary key?
for filename in os.listdir(DATA_DIR):

    filename = pjoin(DATA_DIR, filename)
    with open(filename) as data_file:
        data_string = data_file.read()
        try:
            df = json.loads(data_string)
            probes_names.append(df['Probes'][0]['Name'])
            probes_mean_times.append(df['Probes'][0]['Mean'])
            config_dates.append(df['ITK_MANUAL_BUILD_INFO']['GIT_CONFIG_DATE'])
            itk_versions.append(df['SystemInformation']['ITKVersion'])
        except ValueError:
            print(repr(data_string))


py.sign_in('name', 'API_key')

plt = []

# TODO: compute max from data
yaxis_max = 2

# TODO
# Make the graph nicer: look for a better way to distribute the horizonzal
# axis: daily, weekly?
# Use another format for the xaxis names: either hide them or display the day
# in short ticks; weeks, months in long ticks
# Use short ticks in yaxis?

layout = go.Layout(
    title=probes_names,
    hovermode='closest',
    xaxis=dict(
        title= 'datetime'),
    yaxis=dict(
        title= 'Ellapsed time (s)',
        range=[0, yaxis_max])
)

# TODO
# Create a new plot per module
for dt, m, v in zip(config_dates, probes_mean_times, itk_versions):
    trace = go.Scatter(
        x=dt,
        y=m,
        mode='lines+markers',
        name='ITKv'+v
    )
    plt.append(trace);
    
fig = go.Figure(data=plt, layout=layout)
py.iplot(fig, filename='itk-performance-stats')


# TODO
# Print the slowest execution time per module


PlotlyError: Sign in failed.